# SwinUnet Wildfire Training & Test (Colab)

This notebook trains SwinUnet on a single fold of the WildfireSpreadTS HDF5 dataset, then evaluates on the held-out test split and reports Average Precision (AP) and F1.

**Prerequisites:**
- HDF5 dataset already on Google Drive (from `download_and_convert_dataset.ipynb`)
- **Runtime → Change runtime type → GPU or TPU** (T4/V100 GPU or TPU v2/v3 recommended)
- A [GitHub Personal Access Token](https://github.com/settings/tokens) stored as a Colab secret named `GITHUB_TOKEN` (Colab sidebar → key icon → Add new secret)

## 1. Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Configuration (user-editable)

In [2]:
REPO_ORG   = "amindell11"   # Replace with your GitHub username or organisation
REPO_NAME  = "wildfire-ts-swin"
HDF5_DIR   = "/content/drive/MyDrive/wildfire_dataset/hdf5"
OUTPUT_DIR = "/content/drive/MyDrive/wildfire_runs/fold0"

DATA_FOLD_ID             = 0      # 0–11; which train/val/test year split
N_LEADING_OBSERVATIONS   = 1      # 1 or 5
MAX_EPOCHS               = 100
BATCH_SIZE               = 16
BASE_LR                  = 1e-4
FOCAL_GAMMA              = 2.0
CROP_SIDE_LENGTH         = 128
SEED                     = 42
NUM_WORKERS              = 4      # parallel data workers; 4 is safe on Colab

## 2b. Copy HDF5 files to local disk

Google Drive I/O is ~100× slower than local NVMe. Copying once here keeps the A100 busy instead of waiting on Drive reads. Takes 1–3 min depending on dataset size; skips files already copied.

In [ ]:
import os, shutil, time

_LOCAL_HDF5 = "/content/hdf5"
os.makedirs(_LOCAL_HDF5, exist_ok=True)

_files = sorted(os.listdir(HDF5_DIR))
print(f"Copying {len(_files)} files from Drive → local disk...")
_t0 = time.time()
for _f in _files:
    _src = os.path.join(HDF5_DIR, _f)
    _dst = os.path.join(_LOCAL_HDF5, _f)
    if not os.path.exists(_dst):
        shutil.copy2(_src, _dst)
        print(f"  copied {_f}")
    else:
        print(f"  skipped {_f} (already present)")

print(f"Done in {time.time() - _t0:.0f}s  —  using local path: {_LOCAL_HDF5}")
HDF5_DIR = _LOCAL_HDF5  # all subsequent cells read from local disk

## 3. Clone repo and install dependencies

In [3]:
from google.colab import userdata
_repo_url = f"https://github.com/amindell11/wildfire-ts-swin.git"
!rm -rf /content/wildfire-ts-swin
!git clone $_repo_url /content/wildfire-ts-swin
!pip install -q -r /content/wildfire-ts-swin/requirements.txt
!git -C /content/wildfire-ts-swin log --oneline

Cloning into '/content/wildfire-ts-swin'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 92 (delta 33), reused 85 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 143.04 KiB | 1.83 MiB/s, done.
Resolving deltas: 100% (33/33), done.
4400505 (HEAD -> main, origin/main, origin/HEAD) use tqdm.auto for proper Jupyter/Colab rendering; device-agnostic notebook
d3df338 update log clarity
62ac090 fixes
fc59794 add support for TPU / XLA
894aeda add notebooks
b9fc63a add init.py to recognize datasets as a packages
2e898e2 feat: add AP val and Focal loss for swin training
4aaa44f feat: Average Precision for model evaluation
b63951d update README.md
fd11e4d add code for loading wildfire data
3071415 add swin-unet base code
eef6f9e Initial commit


In [4]:
import sys
sys.path.insert(0, f'/content/{REPO_NAME}')

## 4. Detect accelerator (GPU / TPU / CPU)

In [5]:
!nvidia-smi

Wed Feb 25 19:18:25 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   35C    P0             60W /  400W |    2384MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [6]:
import torch

# Detect the best available device: TPU (XLA) > GPU (CUDA) > CPU
try:
    import torch_xla.core.xla_model as xm
    DEVICE = xm.xla_device()
    print(f"TPU device detected: {DEVICE}")
except ImportError:
    if torch.cuda.is_available():
        DEVICE = torch.device('cuda')
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    else:
        DEVICE = torch.device('cpu')
        print("WARNING: No GPU or TPU detected — running on CPU (will be slow).")

GPU: NVIDIA A100-SXM4-80GB


In [7]:
## 5a. Launch TensorBoard (run this before training; refresh the board while training runs)
import os
os.makedirs(f"{OUTPUT_DIR}/log", exist_ok=True)
%load_ext tensorboard
%tensorboard --logdir {OUTPUT_DIR}/log

Reusing TensorBoard on port 6006 (pid 2149), started 0:42:11 ago. (Use '!kill 2149' to kill it.)

<IPython.core.display.Javascript object>

## 5. Train

In [ ]:
import os
import random
import types
import numpy as np
import torch
import torch.backends.cudnn as cudnn

from config import get_config
from networks.vision_transformer import SwinUnet
from trainer_wildfire import trainer_wildfire
from datasets.wildfire import N_FEATURES_PER_TIMESTEP

if torch.cuda.is_available():
    cudnn.benchmark = True
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

in_chans = N_LEADING_OBSERVATIONS * N_FEATURES_PER_TIMESTEP
extra_opts = [
    'MODEL.SWIN.IN_CHANS', str(in_chans),
    'MODEL.PRETRAIN_CKPT', 'None',
]

args = types.SimpleNamespace(
    data_dir=HDF5_DIR,
    output_dir=OUTPUT_DIR,
    n_leading_observations=N_LEADING_OBSERVATIONS,
    n_leading_observations_test_adjustment=N_LEADING_OBSERVATIONS,
    crop_side_length=CROP_SIDE_LENGTH,
    load_from_hdf5=True,
    data_fold_id=DATA_FOLD_ID,
    max_epochs=MAX_EPOCHS,
    batch_size=BATCH_SIZE,
    base_lr=BASE_LR,
    num_workers=NUM_WORKERS,
    eval_interval=1,
    seed=SEED,
    n_gpu=1,
    focal_gamma=FOCAL_GAMMA,
    cfg=f'/content/{REPO_NAME}/configs/swin_tiny_patch4_window4_128_wildfire.yaml',
    opts=extra_opts,
    zip=False,
    cache_mode='part',
    resume=None,
    accumulation_steps=None,
    use_checkpoint=False,
    amp_opt_level='O1',
    tag=None,
    eval=False,
    throughput=False,
)

config = get_config(args)
os.makedirs(OUTPUT_DIR, exist_ok=True)

net = SwinUnet(config, img_size=config.DATA.IMG_SIZE, num_classes=2).to(DEVICE)
print(f"Model in_chans={in_chans}  (n_leading_observations={N_LEADING_OBSERVATIONS} × 40 features)")
print(f"Model parameters: {sum(p.numel() for p in net.parameters()) / 1e6:.1f}M")

trainer_wildfire(args, net, OUTPUT_DIR, device=DEVICE)

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


=> merge config from /content/wildfire-ts-swin/configs/swin_tiny_patch4_window4_128_wildfire.yaml
SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:2


/usr/local/lib/python3.12/dist-packages/torch/functional.py:505: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4381.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


---final upsample expand_first---
Model in_chans=40  (n_leading_observations=1 × 40 features)
Model parameters: 27.2M


Epochs:   0%|                                                     | 0/100 [00:00<?, ?ep/s]

  Train:   0%|                                                 | 0/309 [00:00<?, ?batch/s]

## 6. Evaluate on test split

In [ ]:
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from datasets.wildfire import WildfireDataset, get_year_split
from utils import compute_binary_metrics, compute_ap

train_years, val_years, test_years = get_year_split(DATA_FOLD_ID)

ckpt_path = f"{OUTPUT_DIR}/best_model.pth"
state_dict = torch.load(ckpt_path, map_location=DEVICE)
if list(state_dict.keys())[0].startswith('module.'):
    state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
net.load_state_dict(state_dict)
net.eval()

db_test = WildfireDataset(
    data_dir=HDF5_DIR,
    included_fire_years=test_years,
    is_train=False,
    stats_years=train_years,
    n_leading_observations=N_LEADING_OBSERVATIONS,
    n_leading_observations_test_adjustment=N_LEADING_OBSERVATIONS,
    crop_side_length=CROP_SIDE_LENGTH,
    load_from_hdf5=True,
)

test_loader = DataLoader(
    db_test, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=(str(DEVICE) == 'cuda'),
)

all_probs, all_preds, all_gts = [], [], []
with torch.no_grad():
    for x_batch, y_batch in tqdm(test_loader, desc="Test"):
        x_batch = x_batch.to(DEVICE)
        logits = net(x_batch)
        probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
        preds = (probs >= 0.5).astype(np.int64)
        gts = y_batch.numpy()
        all_probs.append(probs.flatten())
        all_preds.append(preds.flatten())
        all_gts.append(gts.flatten())

all_probs = np.concatenate(all_probs)
all_preds = np.concatenate(all_preds)
all_gts = np.concatenate(all_gts)

metrics = compute_binary_metrics(all_preds, all_gts)
ap = compute_ap(all_probs, all_gts)

print("\n" + "="*40)
print("Test Results")
print("="*40)
print(f"Test AP   : {ap:.4f}")
print(f"Test F1   : {metrics['f1']:.4f}")
print(f"Precision : {metrics['precision']:.4f}")
print(f"Recall    : {metrics['recall']:.4f}")
print("="*40)

## 7. Note on full 12-fold evaluation

To reproduce the paper's mean ± std AP, run this notebook **12 times** with `DATA_FOLD_ID` set to 0, 1, 2, … 11 (or run a loop in a script). Each fold uses a different train/val/test year split. Average the 12 test AP values to get the reported metric.